In [288]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors
import pickle
import random
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
random.seed(3214)

In [3]:
with open('../data/pickles/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [4]:
df = pd.read_csv('../data/csv/merge_df.csv', index_col=0)
df.head()

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,0.008984,0.031454,0.108398,0.029397,0.009277,0.014918
1,061172.mp3,-692.448303,117.801537,18.185888,5.736947,5.938968,31.851004,1.060427,4.214683,-0.146251,...,-0.198049,0.948205,0.390652,0.080923,0.000357,0.059621,0.351562,0.068662,0.002930,0.058827
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,0.017999,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,0.022331,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,0.010698,0.055768,0.450195,0.098991,0.005859,0.067057


In [5]:
test_idx = list()
while len(test_idx) < 100:
    idx = random.randint(0, df.shape[0] - 1)
    track_id = int(df.loc[idx].track[:-4])
    tags = main_dict[track_id]['all_tags']
    if len(tags) >= 10:
        test_idx.append(idx)

In [6]:
df_test = df.loc[test_idx]
df_test

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
7301,010673.mp3,-626.681091,132.151154,11.753740,45.711430,-0.209040,10.068515,-0.682384,5.395576,-4.435939,...,0.461337,0.281160,0.329500,0.124018,0.027167,0.045374,0.130859,0.031217,0.003418,0.019365
3931,066643.mp3,-687.945374,141.427277,-16.972485,19.092112,6.906140,3.117708,6.967309,-2.045398,-7.208217,...,-0.111801,0.627290,0.380662,0.080697,0.000165,0.061523,0.282227,0.049509,0.002930,0.025951
5176,000194.mp3,-782.241943,178.044266,-29.330959,18.219608,-14.123279,-10.458302,-16.387886,-5.279102,-16.686710,...,0.276664,0.634153,0.264619,0.043619,0.000129,0.038136,0.169922,0.037896,0.001953,0.017058
80,074421.mp3,-623.853394,122.672737,17.257471,4.505549,0.719744,-2.922961,-5.741470,-1.727011,-18.558123,...,0.421106,0.539590,0.264868,0.082399,0.007031,0.040712,0.224121,0.051835,0.009277,0.032241
7903,131553.mp3,-645.512512,74.718880,12.483846,45.145756,38.911320,16.187067,6.199753,0.013115,-3.431449,...,0.100123,0.391042,0.335317,0.112432,0.000625,0.050604,0.271973,0.079668,0.008789,0.049538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708,025603.mp3,-705.887451,168.620865,34.891018,33.937485,15.525203,2.845042,-8.320030,2.128073,10.097413,...,0.422947,1.021024,0.493078,0.113007,0.003961,0.076067,0.147949,0.026009,0.002441,0.020538
3319,075417.mp3,-637.247192,73.643715,17.631123,33.222134,11.122272,9.398943,12.663881,11.582001,10.201411,...,0.267713,0.341603,0.467992,0.120776,0.001821,0.087565,0.348633,0.081037,0.009766,0.044586
5911,125154.mp3,-694.168884,92.781448,59.304184,42.092861,12.254707,19.374973,4.323670,7.782975,-5.282559,...,-0.067733,0.774724,0.482886,0.110169,0.003097,0.075931,0.317871,0.064865,0.002441,0.054606
3341,075436.mp3,-800.051819,241.068558,40.636051,-0.057632,6.397528,-18.648108,-1.818632,3.204657,-0.186070,...,0.923011,1.546881,1.210505,0.112725,0.010022,0.145565,0.138184,0.015291,0.000488,0.012997


In [7]:
df_train = df.loc[list(set(df.index.values) - set(test_idx))]
df_train

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,8.983906e-03,0.031454,0.108398,0.029397,0.009277,0.014918
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,1.799921e-02,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,2.233087e-02,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,1.069842e-02,0.055768,0.450195,0.098991,0.005859,0.067057
5,061592.mp3,-708.504639,136.741913,9.762825,40.804737,17.343578,28.947464,15.307942,17.034895,9.786160,...,-0.039187,0.629073,0.458335,0.132431,6.568079e-11,0.090427,0.515137,0.018989,0.002441,0.039541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,131914.mp3,-654.926941,144.197983,-5.618096,12.815182,-4.743158,4.794647,-0.457318,2.406655,-8.439367,...,0.907090,0.585478,0.172700,0.063363,1.671647e-02,0.027547,0.136230,0.058892,0.012207,0.026086
7978,131772.mp3,-625.237183,158.683304,-10.997794,23.054964,11.636267,-1.839731,-17.972336,-9.967291,-12.986640,...,0.608825,0.747877,0.245719,0.111260,6.102545e-03,0.040364,0.070312,0.020365,0.005859,0.009048
7979,131792.mp3,-655.022644,144.006958,5.970843,14.379925,14.236056,3.561646,-4.041969,-4.800819,-3.666181,...,0.740162,0.707120,0.290256,0.078840,7.514172e-03,0.051776,0.104980,0.032318,0.000488,0.014633
7980,131624.mp3,-788.494507,75.723251,35.248608,28.392780,4.289547,18.946749,-5.460835,10.575089,-8.978883,...,0.130618,0.542250,0.212018,0.045943,9.695412e-04,0.043634,0.321289,0.102740,0.007812,0.053932


In [8]:
features_train = df_train.drop(['track'], 1)
features_test = df_test.drop(['track'], 1)

In [9]:
scaler = StandardScaler()
scaler.fit(features_train)
X_train = scaler.transform(features_train)
X_test = scaler.transform(features_test)

In [10]:
pca = PCA(0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [11]:
X_train.shape

(7885, 103)

In [12]:
X_test.shape

(100, 103)

In [32]:
result = list()
for track in df_train.track.values:
    idx = int(track[:-4])
    if len(main_dict[idx]['all_tags']) >=10:
        result.append(track)
        

In [33]:
len(result)

4016

In [50]:
similarity_1 = list()
similarity_2 = list()
similarity_3 = list()
similarity_4 = list()
similarity_5 = list()
similarity_6 = list()
similarity_7 = list()
similarity_8 = list()
similarity_9 = list()


In [71]:
no_similarity = list()
# while len(similarity_9) < 1000:
for i in range(1000000):
    track_1, track_2 = random.sample(result, 2)
    tags_1 = main_dict[int(track_1[:-4])]['all_tags']
    tags_2 = main_dict[int(track_2[:-4])]['all_tags']
    intersection = set(tags_1) & set(tags_2)
    if len(intersection) == 0:
#         similarity_1.append((track_1, track_2, 0))
        continue
    precision = len(intersection) / len(tags_1)
    recall = len(intersection) / len(tags_2)
    score = 2 * precision * recall / (precision + recall)
    score = round(score, 2)
    if 0.8 <= score < 0.9:
        similarity_9.append((track_1, track_2, score))


In [74]:
all_similarity = similarity_1 +\
similarity_2 +\
similarity_3 +\
similarity_4 +\
similarity_5 +\
similarity_6 +\
similarity_7 +\
similarity_8 +\
similarity_9

In [77]:
all_similarity = list(set(all_similarity))

In [78]:
len(all_similarity)

8231

In [80]:
for tuple in all_similarity:
    with open('../data/tags_similarity.txt', 'a+') as f:
        f.write(f'{tuple[0]} {tuple[1]} {tuple[2]}' + '\n')

In [16]:
with open('../data/tags_similarity.txt', 'r') as f:
    data = f.readlines()

In [17]:
all_similarity = [x.strip('\n').split() for x in data]

In [18]:
all_similarity[0]

['081512.mp3', '081781.mp3', '0.79']

In [21]:
df_train_2 = df_train.reset_index(drop=True)
# df_train_2.reset_index(drop=True, inplace=True)
df_train_2

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,8.983906e-03,0.031454,0.108398,0.029397,0.009277,0.014918
1,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,1.799921e-02,0.079505,0.125000,0.028202,0.000977,0.020876
2,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,2.233087e-02,0.065322,0.267090,0.083095,0.006348,0.036801
3,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,1.069842e-02,0.055768,0.450195,0.098991,0.005859,0.067057
4,061592.mp3,-708.504639,136.741913,9.762825,40.804737,17.343578,28.947464,15.307942,17.034895,9.786160,...,-0.039187,0.629073,0.458335,0.132431,6.568079e-11,0.090427,0.515137,0.018989,0.002441,0.039541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,131914.mp3,-654.926941,144.197983,-5.618096,12.815182,-4.743158,4.794647,-0.457318,2.406655,-8.439367,...,0.907090,0.585478,0.172700,0.063363,1.671647e-02,0.027547,0.136230,0.058892,0.012207,0.026086
7881,131772.mp3,-625.237183,158.683304,-10.997794,23.054964,11.636267,-1.839731,-17.972336,-9.967291,-12.986640,...,0.608825,0.747877,0.245719,0.111260,6.102545e-03,0.040364,0.070312,0.020365,0.005859,0.009048
7882,131792.mp3,-655.022644,144.006958,5.970843,14.379925,14.236056,3.561646,-4.041969,-4.800819,-3.666181,...,0.740162,0.707120,0.290256,0.078840,7.514172e-03,0.051776,0.104980,0.032318,0.000488,0.014633
7883,131624.mp3,-788.494507,75.723251,35.248608,28.392780,4.289547,18.946749,-5.460835,10.575089,-8.978883,...,0.130618,0.542250,0.212018,0.045943,9.695412e-04,0.043634,0.321289,0.102740,0.007812,0.053932


In [25]:
features = list()
target = list()
for tupple in all_similarity:
    idx_1 = df_train_2[df_train_2.track == tupple[0]].index[0]
    idx_2 = df_train_2[df_train_2.track == tupple[1]].index[0]
    arr_1 = X_train[idx_1]
    arr_2 = X_train[idx_2]
    combo = np.hstack((arr_1, arr_2))
    features.append(combo)
    target.append(float(tupple[2]))

In [192]:
with_similar = list()
for idx in main_dict:
    if len(main_dict[idx]['similar_list']) > 0:
        with_similar.append(idx)

In [234]:
features = list()
target = list()
for idx in with_similar:
    track_1 = "{:06d}".format(idx) + '.mp3'
    if track_1 in df_train_2.track.values:
        id_1 = df_train_2[df_train_2.track == track_1].index[0]
        arr_1 = X_train[id_1]
        for pair in main_dict[idx]['similar_list']:
            artist, title, _ = pair
            for idx_2 in main_dict:
                if main_dict[idx_2]['artist_name'] == artist and main_dict[idx_2]['track_title'] == title:
                    track_2 = "{:06d}".format(idx_2) + '.mp3'
                    if track_2 in df_train_2.track.values:
                        id_2 = df_train_2[df_train_2.track == track_2].index[0]
                        arr_2 = X_train[id_2]
                        combo = np.hstack((arr_1, arr_2))
                        features.append(combo)
                        target.append(1)
                        break

In [235]:
len(target)

1905

In [236]:
while len(features) < (1905 * 2):
    pair = random.sample(list(df_train_2.track.values), 2)
    id_1 = int(pair[0][:-4])
    id_2 = int(pair[1][:-4])
    if len(set(main_dict[id_1]['all_genres']) & set(main_dict[id_2]['all_genres'])) == 0:
        idx_1 = df_train_2[df_train_2.track == pair[0]].index[0]
        idx_2 = df_train_2[df_train_2.track == pair[1]].index[0]
        arr_1 = X_train[idx_1]
        arr_2 = X_train[idx_2]
        combo = np.hstack((arr_1, arr_2))
        features.append(combo)
        target.append(0) 

In [237]:
len(features)

3810

In [238]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.1, 
    random_state=123
)

In [69]:
models = {
    'GBR': {
        'model': GradientBoostingRegressor(),
        'params': {
            'loss':['ls', 'lad', 'huber', 'quantile'],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': np.arange(80, 120),
            'subsample': np.arange(0.6, 1.01, 0.1),
#             'min_samples_split': np.arange(20, 70),
#             'min_samples_leaf': np.arange(0.1, .51, 0.1),
            'max_features': np.arange(0.1, 1.01, 0.1),
            'max_depth': np.arange(3, 7)
            
        }
    },
    'LinReg': {
        'model': LinearRegression(),
        'params': {}
    },
    'LGBMR': {
        'model': LGBMRegressor(),
        'params': {
            'boosting_type': ['gbdt', 'dart', 'goss'],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': np.arange(80, 120),
            'subsample': np.arange(0.6, 1.01, 0.1),
            'min_split_gain': np.arange(0.1, 1.01, 0.1),
            
        }
    },
    'SVR': {
        'model': SVR(),
        'params': {
            'kernel': ['rbf', 'linear', 'sigmoid', 'poly'],
            'C': np.arange(0.1, 1.01, 0.1),
            'epsilon': np.arange(0.1, 1.01, 0.1),
            'shrinking': np.arange(0.1, 1.01, 0.1),
            'max_iter': [1000]
        }
    }
}

In [239]:
models = {
    'GBC': {
        'model': GradientBoostingClassifier(),
        'params': {
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': np.arange(80, 120),
            'subsample': np.arange(0.6, 1.01, 0.1),
            'max_features': np.arange(0.1, 1.01, 0.1),
            'max_depth': np.arange(3, 7)
            
        }
    },
    'Logreg': {
        'model': LogisticRegression(),
        'params': {
            'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
            'C': np.arange(0.1, 1.01, 0.1),
            'max_iter': [1000]
        }
    },
    'LGBMC': {
        'model': LGBMClassifier(),
        'params': {
            'boosting_type': ['gbdt', 'dart', 'goss'],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': np.arange(80, 120),
            'subsample': np.arange(0.6, 1.01, 0.1),
            'min_split_gain': np.arange(0.1, 1.01, 0.1),
            
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'kernel': ['rbf', 'linear', 'sigmoid', 'poly'],
            'C': np.arange(0.1, 1.01, 0.1),
            'shrinking': np.arange(0.1, 1.01, 0.1),
            'max_iter': [1000]
        }
    }
}

In [240]:
for model_name in models.keys():
    model = models[model_name]['model']
    params = models[model_name]['params']
    r_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        scoring='roc_auc',
        n_jobs = -1,
        random_state=123,
        error_score='raise' 
    )
    r_search.fit(features_train, target_train)
    y_pred_train = r_search.predict(features_train)
    y_pred_test = r_search.predict(features_test)
#     rmse_train = mean_squared_error(target_train, y_pred_train)
#     rmse_test = mean_squared_error(target_test, y_pred_test)
#     r2_train = r2_score(target_train, y_pred_train)
#     r2_test = r2_score(target_test, y_pred_test)
    print(model)
    print(r_search.best_params_)
    print(classification_report(target_train, y_pred_train))
    print(classification_report(target_test, y_pred_test))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
{'subsample': 0.9999999999999999, 'n_estimators': 104, 'max_features': 0.6, 'max_depth': 6, 'learning_rate': 0.1}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1718
           1       1.00      1.00      1.00      1711

    accuracy             

/home/obi-1-konobe/projects/mag/dp/dp_project/venv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 40 is smaller than n_iter=100. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
{'solver': 'newton-cg', 'max_iter': 1000, 'C': 0.1}
              precision    recall  f1-score   support

           0       0.67      0.66      0.67      1718
           1       0.66      0.68      0.67      1711

    accuracy                           0.67      3429
   macro avg       0.67      0.67      0.67      3429
weighted avg       0.67      0.67      0.67      3429

              precision    recall  f1-score   support

           0       0.64      0.63      0.64       187
           1       0.65      0.66      0.65       194

    accuracy                           0.65       381
   macro avg       0.65      0.65      0.65       381
weighted avg       0.65

/home/obi-1-konobe/projects/mag/dp/dp_project/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
{'shrinking': 0.7000000000000001, 'max_iter': 1000, 'kernel': 'rbf', 'C': 1.0}
              precision    recall  f1-score   support

           0       0.95      0.81      0.87      1718
           1       0.83      0.96      0.89      1711

    accuracy                           0.88      3429
   macro avg       0.89      0.88      0.88      3429
weighted avg       0.89      0.88      0.88      3429

              precision    recall  f1-score   support

           0       0.82      0.63      0.71       187
           1       0.71      0.87      0.78       194

    accuracy                           0.75       381
   macro avg       0.76      0.75      0.75       381
weighted avg       0.76      0.75      0.75       381



In [75]:
gbr = GradientBoostingRegressor(
    subsample=0.9, n_estimators=112, max_features=0.8, max_depth=6, loss='ls', learning_rate=0.1 
)

In [243]:
lgbm = LGBMClassifier(subsample=0.6, n_estimators=119, min_split_gain=0.1, learning_rate=0.1, boosting_type='gbdt')

In [244]:
lgbm.fit(features_train, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=119, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

In [251]:
target_pred = lgbm.predict_proba(features_test)
# target_pred[:,1]

In [246]:
target_train_pred = lgbm.predict(features_train)

In [248]:
print(classification_report(target_test, target_pred))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79       187
           1       0.80      0.79      0.80       194

    accuracy                           0.79       381
   macro avg       0.79      0.79      0.79       381
weighted avg       0.79      0.79      0.79       381



In [79]:
mean_squared_error(target_test, target_pred)

0.0439496096387542

In [80]:
r2_score(target_test, target_pred)

0.2822011674259455

In [81]:
mean_squared_error(target_train, target_train_pred)

0.010734766543312525

In [82]:
r2_score(target_train, target_train_pred)

0.8215008950865157

In [83]:
target_test[432]

0.6

In [84]:
target_pred[432]

0.5410422963872339

In [275]:
neigh = NearestNeighbors(100)
neigh.fit(X_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                 radius=1.0)

In [276]:
def count_good_recommend(query_idx, answer_list, trhold):
    counter = 0
    query_id = int(df_test.iloc[query_idx].track[:-4])
    relevant = [query_id]
    query_tags = main_dict[query_id]['all_tags']
    for idx in answer_list:
        answer_id = int(df_train.iloc[idx].track[:-4])
        answer_tags = main_dict[answer_id]['all_tags']
        score = 0
        i = 0
        for tag in answer_tags:
            if tag in query_tags:
                i += 1
        if i == 0:
            continue
        
        precision = i / len(answer_tags)
        recall = i / len(query_tags)
        score = 2 * precision * recall / (precision + recall)
        if score >= trhold:        
            counter += 1
            relevant.append(answer_id)
    return counter, relevant

In [277]:
def count_ndcg(neighbors, relevant_list):
    dcg = 0
    idcg = 0
    for i in range(len(neighbors)):
        track = int(df_train.iloc[neighbors[i]].track[:-4])
        if track in relevant_list:
            dcg += 1 / math.log(i + 2, 2)
        idcg += 1 / math.log(i + 2, 2)
    ndcg = dcg / idcg
    return ndcg

In [278]:
def rank_by_model(model, neighb, arr_test):
    result = list()
    for idx in neighb:
        arr_2 = X_train[idx]
        combo = np.hstack((arr_test, arr_2))
        score = model.predict_proba([combo])[:,1]
        result.append((idx, score))
    top_10 = sorted(result, key=lambda x: x[1], reverse=True)[:10]
    top_10 = [x[0] for x in top_10]

    return top_10

In [293]:
mean_ap = 0
ndcg = 0
for i in range(100):
    neighbors = neigh.kneighbors([X_test[i]])[1][0]
    level_2 = rank_by_model(lgbm, neighbors, X_test[i])
    relevant_num, relevant_list = count_good_recommend(i, level_2, trhold=0.3)
#     print(relevant_list)
    print(relevant_list[0], [df_train.iloc[x].track[:-4] for x in level_2])
    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(level_2, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

10673 ['087190', '102195', '028484', '085816', '028485', '111378', '001196', '047707', '111397', '127290']
66643 ['056275', '035527', '092949', '116389', '138041', '085817', '126749', '098301', '110166', '091468']
194 ['143218', '032800', '113281', '120309', '016747', '029465', '129439', '016744', '038777', '006330']
74421 ['004080', '142528', '020704', '004075', '072058', '148214', '101112', '107595', '091170', '138406']
131553 ['123835', '045128', '087644', '040842', '055827', '086634', '099260', '112066', '087363', '145260']
24431 ['097961', '056695', '070655', '052035', '056800', '115475', '036961', '126516', '060048', '051260']
144469 ['129092', '075432', '085485', '054665', '006406', '077519', '110927', '131767', '028548', '088860']
87100 ['010439', '075607', '012486', '122623', '082915', '010440', '010385', '116451', '065685', '110927']
69204 ['044821', '100973', '148113', '075378', '044792', '095914', '035549', '085318', '097570', '073760']
27454 ['110690', '080004', '040239', 

40851 ['075393', '100972', '105145', '075399', '095910', '055119', '089815', '071251', '075373', '112066']
1686 ['004080', '004075', '029465', '091468', '138041', '098301', '004078', '087121', '033070', '052040']
73365 ['110690', '087185', '128477', '085836', '075442', '131796', '049473', '080001', '004521', '055549']
5264 ['040985', '035534', '098555', '114399', '004080', '138041', '060994', '036371', '112066', '073770']
19759 ['038781', '126749', '075194', '149103', '029720', '085308', '081084', '035571', '131794', '098301']
41568 ['097589', '085485', '121734', '003532', '139523', '133780', '121739', '148603', '141568', '127289']
24429 ['110690', '085836', '053591', '040239', '113564', '129091', '048864', '122628', '029039', '052449']
86788 ['058542', '015769', '087152', '064006', '112314', '087155', '044802', '048293', '091083', '041825']
152418 ['069764', '149777', '047069', '122398', '109235', '038781', '120188', '004075', '132420', '037368']
64630 ['075379', '078848', '140939', '

In [295]:
level_2

[4701, 2522, 5082, 3518, 6779, 4410, 4793, 3321, 4313, 5558]

In [283]:
mean_ap

0.10899999999999999

In [284]:
mean_ndcg

0.10720239738970358

In [300]:
cos_similarity_mtx = cosine_similarity(X_test, X_train)

In [301]:
mean_ap = 0
ndcg = 0
for i in range(100):
    arr = cos_similarity_mtx[i]
    neighbors = arr.argsort()[-100:][::-1]
    level_2 = rank_by_model(lgbm, neighbors, X_test[i])
    relevant_num, relevant_list = count_good_recommend(i, level_2, trhold=0.3)
#     relevant_num, relevant_list = count_good_recommend(i, neighbors, trhold=0.4)
#     print(relevant_list)
    print(relevant_list[0], [df_train.iloc[x].track[:-4] for x in level_2])
    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(neighbors, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

10673 ['028477', '102195', '106953', '085816', '028485', '111378', '069791', '028482', '047707', '111397']
66643 ['092949', '051269', '067237', '098077', '138041', '013556', '091084', '091468', '142577', '036322']
194 ['143218', '032800', '097587', '127870', '120309', '016747', '053576', '016744', '087189', '105915']
74421 ['004080', '142528', '020704', '072058', '148214', '104009', '101112', '148215', '056520', '085291']
131553 ['023013', '114201', '123835', '144173', '087644', '021404', '040842', '013571', '055827', '086634']
24431 ['097961', '051111', '056695', '070655', '052035', '115475', '036961', '126516', '131326', '051260']
144469 ['075432', '012486', '052954', '091897', '006407', '006406', '077519', '110927', '114396', '127192']
87100 ['093364', '010439', '075607', '010440', '010385', '123488', '116451', '133449', '051776', '057628']
69204 ['044821', '075378', '070654', '084096', '070770', '141593', '070768', '139937', '035549', '110771']
27454 ['080004', '040239', '075442', 

73365 ['110690', '092574', '086039', '085836', '013708', '049473', '004521', '052449', '055186', '081576']
5264 ['112065', '021404', '075379', '055285', '098555', '126047', '099260', '087237', '060994', '098551']
19759 ['040239', '085836', '038781', '075194', '149103', '057819', '100497', '154413', '124912', '029720']
41568 ['056519', '097589', '083440', '115176', '049849', '129188', '126283', '116871', '145476', '056797']
24429 ['110690', '069822', '053591', '133452', '040239', '035549', '124870', '048864', '122628', '045392']
86788 ['108502', '040986', '058542', '087152', '064006', '112314', '021895', '096935', '054624', '054626']
152418 ['069764', '040235', '122398', '009918', '038817', '040232', '014590', '004836', '038776', '084158']
64630 ['140939', '011922', '054470', '073768', '114061', '026008', '043857', '071231', '107129', '025605']
85832 ['110690', '075442', '069791', '085836', '092546', '087185', '080001', '046024', '052449', '116116']
75317 ['075379', '098556', '129887', 

In [291]:
mean_ap

0.12099999999999995

In [292]:
mean_ndcg

0.013384470428357782

[6645, 4701, 7183, 3518, 4410, 7236, 3979, 1258, 7490, 7097]

In [304]:
a = lgbm.predict_proba(X_train)

ValueError: Number of features of the model must match the input. Model n_features_ is 206 and input n_features is 103 

87097 ['081660', '132566', '108024', '108022', '133970', '024217', '108017', '061007', '132448', '142098']

87097 ['126559', '087103', '108885', '087101', '126321', '121987', '142098', '125657', '108024', '086481']

87097 ['111222', '087103', '086486', '133561', '065038', '064994', '028274', '077522', '087099', '098237']

87097 ['111222', '087103', '086486', '081803', '133561', '065038', '133580', '116101', '084201', '126557']

In [103]:
df_test_2 = df_test.reset_index(drop=True)


In [302]:
track_1 = '087097.mp3'
track_2 = '126557.mp3'

In [303]:
idx_1 = df_test_2[df_test_2.track == track_1].index[0]
idx_2 = df_train_2[df_train_2.track == track_2].index[0]
arr_1 = X_test[idx_1]
arr_2 = X_train[idx_2]
combo = np.hstack((arr_1, arr_2))
lgbm.predict_proba([combo])[:,1]

array([0.37847317])

In [176]:
tags_1 = main_dict[int(track_1[:-4])]['all_tags']
tags_2 = main_dict[int(track_2[:-4])]['all_tags']
intersection = set(tags_1) & set(tags_2)

precision = len(intersection) / len(tags_1)
recall = len(intersection) / len(tags_2)
score = 2 * precision * recall / (precision + recall)
score = round(score, 2)
score

1.0

In [177]:
tags_1

['idm',
 'ambient',
 'electronic',
 'instrumental',
 'chillout',
 'electronica',
 'post-rock',
 'classic',
 'electro',
 'soundtrack']

In [178]:
tags_2

['idm',
 'ambient',
 'electronic',
 'instrumental',
 'chillout',
 'electronica',
 'post-rock',
 'classic',
 'electro',
 'soundtrack']

In [182]:
result = list()
for idx in main_dict:
    if len(main_dict[idx]['similar_list']) > 0:
        result.append(idx)

In [191]:
len(result)

1138

In [211]:
len(set(main_dict[534]['all_genres']) & set(main_dict[540]['all_genres']))

1

In [190]:
main_dict[540]

{'artist_name': 'Charles Manson',
 'track_title': 'The Black Pirate',
 'artist_tags': [('folk', '100'),
  ('singer-songwriter', '58'),
  ('psychedelic', '54'),
  ('experimental', '36'),
  ('misunderstood geniuses', '27'),
  ('acoustic', '20'),
  ('psychedelic folk', '12'),
  ('serial killer', '9'),
  ('american', '8'),
  ('acid folk', '8')],
 'track_tags': [('counter-culture', '100')],
 'similar_tracks': [('Charles Manson', 'Brother Gun', 1.0),
  ('Charles Manson', 'Angels Fear To Tread', 0.82),
  ('Tim Buckley', 'No Man Can Find The War', 0.311),
  ('Jasmina Maschina', 'Love You More & More', 0.012),
  ('Jasmina Maschina', 'Joyrider', 0.012),
  ('Ween', 'Duke of Denim', 0.012),
  ('Damien Youth', 'Bride of the Asylum', 0.011),
  ('Donovan', 'Cosmic Wheels', 0.011),
  ('Damien Youth', 'The Girl With the Serpent Eyes', 0.011),
  ('Damien Youth', 'Fran (Ex Left Banke - The Beckies Cover)', 0.011),
  ('Damien Youth', 'The Telepath (Unreleased Track)', 0.011),
  ('Damien Youth', 'Test Tube

In [189]:
for idx in main_dict:
    if main_dict[idx]['artist_name'] == 'Charles Manson' and main_dict[idx]['track_title'] == 'The Black Pirate':
        print(idx)
        break

540


In [111]:
df_test_2[df_test_2.track == '087097.mp3']

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
47,087097.mp3,-815.897278,330.361633,-24.059116,-18.602432,20.005129,-4.015804,-11.668517,-7.022644,-0.156837,...,2.436082,2.976102,0.252494,0.05244,0.012415,0.031541,0.037598,0.018503,0.005859,0.005123


In [109]:
df_test_2

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,010673.mp3,-626.681091,132.151154,11.753740,45.711430,-0.209040,10.068515,-0.682384,5.395576,-4.435939,...,0.461337,0.281160,0.329500,0.124018,0.027167,0.045374,0.130859,0.031217,0.003418,0.019365
1,066643.mp3,-687.945374,141.427277,-16.972485,19.092112,6.906140,3.117708,6.967309,-2.045398,-7.208217,...,-0.111801,0.627290,0.380662,0.080697,0.000165,0.061523,0.282227,0.049509,0.002930,0.025951
2,000194.mp3,-782.241943,178.044266,-29.330959,18.219608,-14.123279,-10.458302,-16.387886,-5.279102,-16.686710,...,0.276664,0.634153,0.264619,0.043619,0.000129,0.038136,0.169922,0.037896,0.001953,0.017058
3,074421.mp3,-623.853394,122.672737,17.257471,4.505549,0.719744,-2.922961,-5.741470,-1.727011,-18.558123,...,0.421106,0.539590,0.264868,0.082399,0.007031,0.040712,0.224121,0.051835,0.009277,0.032241
4,131553.mp3,-645.512512,74.718880,12.483846,45.145756,38.911320,16.187067,6.199753,0.013115,-3.431449,...,0.100123,0.391042,0.335317,0.112432,0.000625,0.050604,0.271973,0.079668,0.008789,0.049538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,025603.mp3,-705.887451,168.620865,34.891018,33.937485,15.525203,2.845042,-8.320030,2.128073,10.097413,...,0.422947,1.021024,0.493078,0.113007,0.003961,0.076067,0.147949,0.026009,0.002441,0.020538
96,075417.mp3,-637.247192,73.643715,17.631123,33.222134,11.122272,9.398943,12.663881,11.582001,10.201411,...,0.267713,0.341603,0.467992,0.120776,0.001821,0.087565,0.348633,0.081037,0.009766,0.044586
97,125154.mp3,-694.168884,92.781448,59.304184,42.092861,12.254707,19.374973,4.323670,7.782975,-5.282559,...,-0.067733,0.774724,0.482886,0.110169,0.003097,0.075931,0.317871,0.064865,0.002441,0.054606
98,075436.mp3,-800.051819,241.068558,40.636051,-0.057632,6.397528,-18.648108,-1.818632,3.204657,-0.186070,...,0.923011,1.546881,1.210505,0.112725,0.010022,0.145565,0.138184,0.015291,0.000488,0.012997


In [84]:
df_train[df_train.track == '081512.mp3'].drop(columns=['track']).values

array([[-7.74647705e+02,  1.26531204e+02,  1.43112164e+01,
         1.14453869e+01, -8.95931244e+00, -9.55889320e+00,
        -1.35218897e+01,  2.28147745e+00,  1.78788033e+01,
         7.60553885e+00,  6.46068764e+00,  5.74957466e+00,
         2.95985079e+00,  4.32843555e+03,  1.02788281e+03,
         1.08554832e+02,  9.14352112e+02,  1.19996819e+02,
         1.40324879e+01, -1.35242832e+00,  1.88848228e+01,
         1.13695312e+04,  1.68837732e+03,  8.61328125e+01,
         2.64714722e+03,  9.10147858e+00,  2.54628015e+00,
         4.16384824e-02,  1.58443892e+00,  3.64557415e-01,
         7.18303695e-02,  6.19487313e-04,  7.85310268e-02,
         3.59863281e-01,  5.66855259e-02,  2.44140625e-03,
         6.01763166e-02, -8.02175903e+02,  1.36923691e+02,
         2.09778728e+01,  2.03402443e+01,  2.40463424e+00,
         6.60363150e+00,  4.70998526e+00,  1.70874729e+01,
         2.93335152e+01,  1.52756615e+01,  3.09140229e+00,
        -4.18995047e+00, -4.08248231e-03,  3.83288086e+0

In [39]:
a = main_dict[84096]['all_tags']
b = main_dict[136994]['all_tags']
set(a) & set(a)

{'abstract',
 'abstract hip hop',
 'braindance',
 'breakbeat',
 'breakcore',
 'drum & bass',
 'dubstep',
 'electronic',
 'electronica',
 'glitch',
 'idm',
 'under 2000 listeners'}